In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ain-space/gaia/model-a2gm8521:v0', type='model')
artifact_dir = artifact.download()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bernddoser. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [3]:
import torch
from spherinator.models import (
    ConvolutionalDecoder1D,
    ConvolutionalEncoder1D,
    VariationalAutoencoderPure,
)

encoder = ConvolutionalEncoder1D(input_dim=344, output_dim=3)
decoder = ConvolutionalDecoder1D(input_dim=3, output_dim=344)
model = VariationalAutoencoderPure(encoder=encoder, decoder=decoder)

ckpt = "artifacts/model-a2gm8521:v0/model.ckpt"
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")
model.load_state_dict(checkpoint["state_dict"])
model.eval()

export_options = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx = torch.onnx.dynamo_export(model.variational_encoder, torch.randn(1, 1, 344, device="cpu"), export_options=export_options)
onnx.save("vae_encoder.onnx")

onnx = torch.onnx.dynamo_export(model.decoder, torch.randn(1, 3, device="cpu"), export_options=export_options)
onnx.save("vae_decoder.onnx")

/local_data/doserbd/git/Spherinator/.venv/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/local_data/doserbd/git/Spherinator/.venv/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/local_data/doserbd/git/Spherinator/.venv/lib/python3.10/site-packages/torch/onnx/_internal/_exporter_legacy.py:116: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/local_data/doserbd/git/Spherinator/.venv/lib/python3.10/site-packages/torch/onnx/_internal/fx/pas